BIBLIOTEACAS

In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

import pandas as pd

import time

import csv

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

import tkinter as tk

from pandasgui import show

from datetime import date

FUNÇÃO PARA ROLAR A PAGINA ATÉ O FIM

In [2]:
from selenium.webdriver.common.keys import Keys

def scroll_to_bottom(driver):
    # define o último tamanho da página como 0 para iniciar o loop
    last_height = 0
    # enquanto o tamanho da página mudar
    while True:
        # rola a página até o final
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # espera um pouco para que a página carregue
        time.sleep(2)
        # obtém o novo tamanho da página
        new_height = driver.execute_script("return document.body.scrollHeight")
        # se o novo tamanho da página for o mesmo que o último, a página não mudou, então saia do loop
        if new_height == last_height:
            break
        # atualize o último tamanho da página
        last_height = new_height
    # role para o topo da página (opcional)
    driver.execute_script("window.scrollTo(0, 0);")

FUNÇÃO PARA PEGAR PRODUTOS DA PÁGINA

In [3]:
def Coleta(driver):

    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    
    time.sleep(1)

    scroll_to_bottom(driver)
    
    descricao = driver.find_elements(By.CLASS_NAME, "product-card__description")
    preco = driver.find_elements(By.CLASS_NAME, "product-card__price")

    produtos = {}

    for i in range(len(descricao)):
        produtos[descricao[i].text] = preco[i].text
    
    #ImportaCSV(produtos)
    '''
    print("Produtos disponíveis:")
    for desc, preco in produtos.items():
        if desc != "":
            print(f"{desc}: {preco}")
    '''
    
    return produtos

FUNÇÃO PARA IMPORTA DATAFRAME PARA UM ARQUIVO EM EXCEL

In [4]:
def ImportaCSV(dados):

    num_linhas = len(dados['nome'])
    num_colunas = len(dados)

    print(f'O dicionário tem {num_linhas} linhas e {num_colunas} colunas.')

    # Converter o dicionário em um DataFrame
    df = pd.DataFrame.from_dict(dados)

    # Salvar o DataFrame em um arquivo do Excel
    nome_arquivo = "exemplo.xlsx"
    #df.to_excel(nome_arquivo, index=False)

FUNÇÃO PARA SALVAR EM UM DATAFRAME

In [5]:
def salvarDataframe(meu_dicionario):

    #Criando DataFrame
    dataProdutos = pd.DataFrame(columns=['descrição', 'valor'])
    i= 0
    for chave, valor in meu_dicionario.items():
        valorExtraido = valor.split('\n')

        #Adicionando no dataframe
        dataProdutos = pd.concat([dataProdutos, pd.DataFrame({'descrição': [chave], 'valor': [valorExtraido[0]]})], ignore_index=True)
    
    return dataProdutos

FUNÇÃO: COLETAR TODOS OS PRODUTOS DO SITE

In [6]:
def limpaGeral(url):
    service = Service('ADD-YOUR-chromedriver.exe-PATH')
    driver = webdriver.Chrome(service=service)

    driver.get(url)

    lista_de_links =[] 

    time.sleep(2)
    nomeMercado  = driver.find_element(By.CLASS_NAME, "market-header__title").text
    lista  = driver.find_elements(By.CLASS_NAME, "aisle-menu__item__link")

    print(nomeMercado)

    for indice, element in enumerate(lista):
        if indice >= 3 : #and indice <4 :
            print(f'{element.text} e seu link : "{element.get_attribute("href")}" ')
            lista_de_links.append(element.get_attribute("href")) #adiciona links a lista
            #linkAchado = element.get_attribute("href")

    dfFinal = pd.DataFrame(columns=[])

    for link in lista_de_links:
        driver.get(link)
        lista_produtos = Coleta(driver)
        dfProdutos = salvarDataframe(lista_produtos)
        dfFinal = pd.concat([dfFinal, salvarDataframe(lista_produtos)]).reset_index(drop=True)
        time.sleep(1)

    return dfFinal,nomeMercado
        
    #show(dfFinal) #Mostrar dataframe extraido

    #data_atual = date.today()
    #data_formatada = data_atual.strftime("%d-%m-%Y")

    #dfFinal.to_excel(f'-{nomeMercado}_{data_formatada}.xlsx', index=False) #Convertendo excel

EXECUTÁVEL

In [7]:

links = ['https://www.ifood.com.br/delivery/manaus-am/supermercado-veneza---flores-flores/2da7761c-11c1-4fcf-bbc1-bd2c083f58ee',        
         'https://www.ifood.com.br/delivery/manaus-am/patio-gourmet-adrianopolis-adrianopolis/d68fbb1e-7ab7-4808-b6f9-d8000a97117c', 
         'https://www.ifood.com.br/delivery/manaus-am/carrefour-hiper---manaus-flores-flores/5a3e1f8b-836c-4ae4-9d80-a9dd5faaf55d',
         'https://www.ifood.com.br/delivery/manaus-am/super-nova-era---torres-novo-aleixo/f1e6a3fa-8088-423a-947d-665532bb3faa',
         'https://www.ifood.com.br/delivery/manaus-am/supermercado-rodrigues---vieira-alves-nossa-senhora-das-gracas/912e9262-6b4c-492a-8e3f-f2a6b3f02e89',
         'https://www.ifood.com.br/delivery/manaus-am/db-supermercados-adrianopolis-adrianopolis/ba274260-9144-4833-83b1-ec57643bd66d']


data_atual = date.today()
data_formatada = data_atual.strftime("%d-%m-%Y")

writer = pd.ExcelWriter(f'Coleta_{data_formatada}.xlsx', engine='openpyxl')

for link in links:
    df, nomeMercado = limpaGeral(link)
    df.to_excel(writer, sheet_name=f'{nomeMercado[:31]}', index=False)

writer.book.save(f'Coleta_{data_formatada}.xlsx')
    

Supermercado Veneza - Flores
Bazar e Utilidades e seu link : "https://www.ifood.com.br/delivery/manaus-am/supermercado-veneza---flores-flores/2da7761c-11c1-4fcf-bbc1-bd2c083f58ee?corredor=62869bed-b8a4-4116-8e5c-608534e36f94&originArea=aisleMenu" 
Bebidas e seu link : "https://www.ifood.com.br/delivery/manaus-am/supermercado-veneza---flores-flores/2da7761c-11c1-4fcf-bbc1-bd2c083f58ee?corredor=ef27c0d9-2d80-480d-97e4-95cef75400eb&originArea=aisleMenu" 
Bebidas Alcoólicas e seu link : "https://www.ifood.com.br/delivery/manaus-am/supermercado-veneza---flores-flores/2da7761c-11c1-4fcf-bbc1-bd2c083f58ee?corredor=c1fb8989-475c-464b-82b3-6988de63da4f&originArea=aisleMenu" 
Biscoitos e Salgadinhos e seu link : "https://www.ifood.com.br/delivery/manaus-am/supermercado-veneza---flores-flores/2da7761c-11c1-4fcf-bbc1-bd2c083f58ee?corredor=1b2e34ff-0d1c-49fb-9ee8-b2933ad95d9c&originArea=aisleMenu" 
Carnes, Aves e Peixes e seu link : "https://www.ifood.com.br/delivery/manaus-am/supermercado-veneza--

: 

: 